In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as  transforms
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchsummary import summary
from torch.autograd import Variable
import time as time
import numpy as np
from sklearn.metrics import accuracy_score

In [7]:
train = datasets.CIFAR100(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
test = datasets.CIFAR100(root='./data/', train=False, transform=transforms.ToTensor(), download=True)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
import alexnet
import augment

In [1]:
%load_ext tensorboard

In [2]:
%tensorboard --logdir log

In [4]:
# 显示网络参数量
def Init_net():
    model = alexnet.AlexNet()
    data_input = Variable(torch.randn(8,3,32,32))
    print(data_input.size())
    model(data_input)
    print(summary(model,(3,32,32)))

In [5]:
Init_net()

torch.Size([8, 3, 32, 32])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 16, 16]           7,296
              ReLU-2           [-1, 96, 16, 16]               0
 LocalResponseNorm-3           [-1, 96, 16, 16]               0
         MaxPool2d-4             [-1, 96, 8, 8]               0
            Conv2d-5            [-1, 256, 8, 8]         614,656
              ReLU-6            [-1, 256, 8, 8]               0
 LocalResponseNorm-7            [-1, 256, 8, 8]               0
         MaxPool2d-8            [-1, 256, 4, 4]               0
            Conv2d-9            [-1, 384, 4, 4]         885,120
             ReLU-10            [-1, 384, 4, 4]               0
           Conv2d-11            [-1, 384, 4, 4]       1,327,488
             ReLU-12            [-1, 384, 4, 4]               0
           Conv2d-13            [-1, 384, 4, 4]       1,327,488
            

### 准备数据

In [9]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('log')

In [37]:
for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

In [163]:
%tensorboard --logdir log

In [122]:
model = alexnet.AlexNet()
datasets={'train':train,'test':test}
dataloaders = { x:DataLoader(datasets[x],batch_size=100,shuffle=True) for x in ['train','test']}
optimizerAdam = torch.optim.Adam(model.parameters(), lr=0.001)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [86]:
max(a)

4

In [130]:
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
_, preds = torch.max(output,1)

In [131]:
input

tensor([[ 0.2515,  0.4354,  0.1378],
        [-1.2044, -1.2540,  0.5142]])

In [44]:
test

Dataset CIFAR100
    Number of datapoints: 10000
    Split: test
    Root Location: ./data/
    Transforms (if any): ToTensor()
    Target Transforms (if any): None

In [100]:
train[0][0]

tensor([[[1.0000, 1.0000, 1.0000,  ..., 0.7647, 0.8314, 0.7137],
         [1.0000, 0.9961, 0.9961,  ..., 0.6667, 0.6314, 0.5725],
         [1.0000, 0.9961, 1.0000,  ..., 0.7412, 0.6510, 0.4745],
         ...,
         [0.5804, 0.5569, 0.5490,  ..., 0.1176, 0.2549, 0.2980],
         [0.4784, 0.4706, 0.4941,  ..., 0.0863, 0.3804, 0.5529],
         [0.3412, 0.3451, 0.3961,  ..., 0.1333, 0.4118, 0.5412]],

        [[1.0000, 1.0000, 1.0000,  ..., 0.8039, 0.8784, 0.7608],
         [1.0000, 0.9961, 0.9961,  ..., 0.6902, 0.6588, 0.6039],
         [1.0000, 0.9961, 1.0000,  ..., 0.7804, 0.6980, 0.5216],
         ...,
         [0.7255, 0.7137, 0.7020,  ..., 0.0667, 0.2431, 0.3020],
         [0.6157, 0.6078, 0.6275,  ..., 0.0627, 0.4392, 0.6314],
         [0.4784, 0.4784, 0.5255,  ..., 0.1412, 0.5216, 0.6784]],

        [[1.0000, 1.0000, 1.0000,  ..., 0.7569, 0.8000, 0.6549],
         [1.0000, 0.9961, 0.9961,  ..., 0.5882, 0.5098, 0.4431],
         [1.0000, 0.9961, 1.0000,  ..., 0.6627, 0.5098, 0.

In [101]:
dataloader_train = DataLoader(train, batch_size=100, shuffle=False)
dataloader_test = DataLoader(test, batch_size=100, shuffle=False)

In [131]:
def train_model(model,num_epochs=500):
    optimizerAdam = torch.optim.Adam(model.parameters(), lr=0.01)
    lossFunc = nn.functional.binary_cross_entropy
    since = time.time()
    best_acc = 0.0
    losses = []
    accs = []
    for epoch in range(num_epochs):
        print('Epoch{}/{}'.format(epoch,num_epochs-1))
        
        for step,(inputs,labels) in enumerate(dataloader_train):
            model.train()
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels_onehot =(torch.tensor(np.eye(100)[labels.cpu()],dtype=torch.float32)).to(device)
            outputs = model(inputs)
            y = nn.Softmax(dim=1)(outputs)
            preds = torch.max(outputs.data,1)[1].cpu().numpy() ##取每行最大
            train_loss = lossFunc(y, labels_onehot)

            optimizerAdam.zero_grad()
            train_loss.backward()
            optimizerAdam.step()
            losses.append(train_loss)

            if step % 100 == 0:
                test_accuracy = 0
                for step2,(inputs_test,labels_test) in enumerate(dataloader_test):
                    model.eval()
                    inputs_test = inputs_test.to(device)
                    labels_test = labels_test.to(device)
                    output_test = model(inputs_test)
                    y_test = nn.Softmax(dim=1)(output_test)
                    _, pred_test = torch.max(y_test.data, 1)
                    test_accuracy = accuracy_score(labels_test, pred_test)
                    
                accs.append(test_accuracy)
                print('step:',step)
                print('test accuracy:', test_accuracy)
                if test_accuracy > best_acc:
                    best_acc = test_accuracy
  
                
    for i in range(len(losses)):
        writer.add_scalar("Loss/train",losses[i],i)  
    for j in range(len(accs)):
        writer.add_scalar("Accuracy/test",accs[i],i)  
    
    time_elapsed = time.time() - since
    print('Trainning complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60 , time_elapsed % 60
    ))
    print('Best test Acc: {:4f}'.format(best_acc))

    return model

In [132]:
baseline = train_model(model,num_epochs=500)

Epoch0/499
step: 0
test accuracy: 0.01
step: 100
test accuracy: 0.02
step: 200
test accuracy: 0.0
step: 300
test accuracy: 0.0
step: 400
test accuracy: 0.01
Epoch1/499
step: 0
test accuracy: 0.0
step: 100
test accuracy: 0.0
step: 200
test accuracy: 0.0
step: 300
test accuracy: 0.0
step: 400
test accuracy: 0.01
Epoch2/499
step: 0
test accuracy: 0.0
step: 100
test accuracy: 0.0
step: 200
test accuracy: 0.0
step: 300
test accuracy: 0.0
step: 400
test accuracy: 0.01
Epoch3/499
step: 0
test accuracy: 0.0
step: 100
test accuracy: 0.0
step: 200
test accuracy: 0.0
step: 300
test accuracy: 0.0
step: 400
test accuracy: 0.01
Epoch4/499
step: 0
test accuracy: 0.0
step: 100
test accuracy: 0.0
step: 200
test accuracy: 0.0
step: 300
test accuracy: 0.0
step: 400
test accuracy: 0.01
Epoch5/499
step: 0
test accuracy: 0.0
step: 100
test accuracy: 0.0
step: 200
test accuracy: 0.0
step: 300
test accuracy: 0.0
step: 400
test accuracy: 0.01
Epoch6/499
step: 0
test accuracy: 0.0
step: 100
test accuracy: 0.0
s

KeyboardInterrupt: 

In [125]:
lossFunc = nn.functional.binary_cross_entropy
since = time.time()
best_acc = 0.0
loss=[]
accu=[]
num=1
for epoch in range(num):
    print('Epoc{}/{}'.format(epoch,num-1))
    print('-'*10)
    # epoch一次完成后切换到测试phase

    for step,(inputs,labels) in enumerate(dataloader):
        model.train()
        print(step)
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels_onehot =(torch.tensor(np.eye(100)[labels],dtype=torch.float32)).to(device)
        outputs = model(inputs)
        y = nn.Softmax(dim=1)(outputs)
        _, preds = torch.max(y.data,1) ##取每行最大
        print(preds)
        train_loss = lossFunc(y, labels_onehot)

        optimizerAdam.zero_grad()
        train_loss.backward()
        optimizerAdam.step()
        loss.append(train_loss)
        if step % 100 == 0:
            for step2,(inputs_test,labels_test) in enumerate(da):
                
                output = model(inputs_test)
                _, pred = torch.max(output, 1)
            test_accuracy = accuracy_score(labels_test, pred)
            accu.append(test_accuracy)
            print('test accuracy:', test_accuracy)

Epoc0/0
----------
0
tensor([59, 14, 14, 59, 14, 61, 14, 14, 13, 61, 14, 59, 14, 14, 59, 14, 61, 14,
        14, 59, 61, 59, 61, 59, 59, 61, 14, 14, 14, 14, 59, 97, 59, 14, 14, 61,
        14, 14, 14, 14, 14, 14, 14, 14, 14, 61, 14, 14, 14, 97, 14, 14, 14, 14,
        61, 13, 61, 59, 61, 59, 14, 14, 14, 14, 14, 14, 14, 61, 61, 14, 61, 14,
        14, 61, 14, 13, 14, 13, 59, 59, 61, 61, 14, 14, 14, 14, 14, 14, 14, 14,
        14, 53, 59, 14, 14, 14, 59, 14, 59, 59])
test accuracy: 0.0102
1
tensor([ 6, 61, 61, 61,  6, 61, 61,  6,  6, 61, 61,  6,  6,  6, 61, 61,  6,  6,
         6, 61,  6, 71,  6, 61, 61,  6, 61,  6, 71,  6,  6, 61, 61,  6, 61, 61,
         6, 61, 61,  6, 61,  6,  6, 61, 71, 61, 61,  6,  6, 61, 61,  6,  6,  6,
         6, 61,  6,  6,  6,  6,  6, 61, 61,  6, 61,  6,  6, 71, 61, 61, 61, 61,
         6, 61,  6, 61,  6,  6, 61, 61, 61,  6, 61,  6,  6, 61,  6, 61, 61,  6,
        61,  6,  6,  6,  6,  6, 61,  6,  6,  6])
2
tensor([71, 61, 61, 61, 71, 61, 71, 61, 61, 61, 61, 71,

KeyboardInterrupt: 

In [ ]:
def train_model(model,num_epochs=5):
    since = time.time()
    best_acc = 0.0
    losses = []
    accs = []
    for epoch in range(num_epochs):
        print('Epoc{}/{}'.format(epoch,num_epochs-1))
        print('-'*10)
        # epoch一次完成后切换到测试phase
        for phase in ['train','test']:
            if phase == 'train':
                model.train()   # 切换到train mode
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for inputs,labels in enumerate(dataloaders[phase]):
                print(inputs.shape)
                inputs = inputs.to(device)
                labels_onehot =(torch.tensor(np.eye(100)[labels],dtype=torch.float32)).to(device)
                optimizerAdam.zero_grad() # 梯度清零
                # forwaard
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    print(outputs)
                    y = nn.Softmax(dim=1)(outputs)
                    loss = torch.nn.functional.binary_cross_entropy(y, labels_onehot)
                    _, preds = torch.max(y.data,1) ##取每行最大
                    print(y,preds)
                    acc = accuracy_score(labels,preds,normalize=False)
                    print('loss:{}, accu:{}'.format(loss,acc))

                    # backward + optimize only if in trainning phase
                    if phase == 'train':
                        loss.backward()
                        optimizerAdam.step()
                        losses.append(loss.item())
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += acc
                
#             if phase == 'train':
#                 epoch_loss = running_loss / 50
#                 epoch_acc = running_corrects.double() / 50
#                 print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
                
#             else:
#                 epoch_loss = running_loss / 50
#                 epoch_acc = running_corrects.double() / 50
#                 print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
               
#                 if epoch_acc > best_acc:
#                     best_acc = epoch_acc 
                    
#                 if phase == "test":
#                     if acc > best_acc:
#                         best_acc = acc
#                     accs.append(acc)
                    
#                 print('loss:{}, accu:{}'.format(sum(losses),acc))
                
                
    for i in range(len(losses)):
        writer.add_scalar("train_loss",losses[i],i)  
    for j in range(len(accs)):
        writer.add_scalar("test_acc",accs[i],i)  
    
    time_elapsed = time.time() - since
    print('Trainning complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60 , time_elapsed % 60
    ))
    print('Best test Acc: {:4f}'.format(best_acc))

    return model

In [4]:
train_model(model,num_epochs=1)

NameError: name 'model' is not defined

In [5]:
def augmentation(augmentor,aug_batch):
    if augmentor == 'cutout':
        cutout = augment.Cutout(1,16)
        img_batch_cutout,labels = cutout(aug_batch)
        return img_batch_cutout,labels
    if augmentor == 'mixup':
        mixupor = augment.Mixup(0.5) ##设alpha
        img_batch_mixup, labels = mixupor(aug_batch)
        return img_batch_mixup, labels
    if augmentor == 'cutmix':
        cutmixer = augment.CutMix(0.5) ##设alpha
        img_batch_mixup = cutmixer(aug_batch)
        return img_batch_mixup, labels

In [6]:
def train_model_augment(model,criterion,optimizer,augmentor,num_epochs=1):
    since = time.time()
    best_acc = 0.0
    losses = []
    accs = []
    for epoch in range(num_epochs):
        print('Epoc{}/{}'.format(epoch,num_epochs-1))
        print('-'*10)
        # epoch一次完成后切换到测试phase
        for phase in ['train','test']:
            if phase == 'train':
                model.train()   
            else:
                model.eval()
            
            for ori_inputs,ori_labels in tqdm(dataloaders[phase]):
                if phase == 'train':
                    
                    aug_batch = []
                    for i in range(n):
                        tup = (ori_inputs[i],ori_labels[i])
                        aug_batch.append(tup)
                    if augmentor == 'cutout':
                        train_batch, train_batch_labels = augmentation(augmentor,aug_batch)
                    if augmentor == 'cutmix' or augmentor == 'mixup':
                        train_batch, train_batch_labels = augmentation(augmentor,aug_batch)
                    inputs = train_batch.to(device)
                    labels = train_batch_labels.to(device)
                    
                else:
                    inputs = ori_inputs.to(device)
                    labels =(torch.tensor(np.eye(100)[ori_labels],dtype=torch.float32)).to(device)
                    
                optimizer.zero_grad() # 梯度清零
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    y = torch.nn.Softmax(dim=-1)(outputs)
                    loss = torch.nn.functional.binary_cross_entropy(y, labels)
                    _, preds =torch.max(y,1) ##取每行最大
                    acc = accuracy_score(preds, ori_labels)
                    print('loss:{}, accu:{}'.format(loss,acc))

                    # backward + optimize only if in trainning phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        losses.append(loss.item())
                    
                if phase == "test":
                    if acc > best_acc:
                        best_acc = acc
                    accs.append(acc)
                
                
    for i in range(len(losses)):
        writer.add_scalar("train_loss",losses[i],i)  
    for j in range(len(accs)):
        writer.add_scalar("test_acc",accs[i],i)  
    
    time_elapsed = time.time() - since
    print('Trainning complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60 , time_elapsed % 60
    ))
    print('Best test Acc: {:4f}'.format(best_acc))

    return model

In [7]:
train_model_augment(model,loss_func,optimizerAdam,'cutout',1)

Epoc0/0
----------


  0%|                                                   | 0/500 [00:00<?, ?it/s]

100
<class 'torch.Tensor'> 100


  0%|                                                   | 0/500 [00:00<?, ?it/s]


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)